Why do programmers not like nature? *Too many bugs and no documentation...*

# Pre-Processing
| Dependency | Description | Link |
|------------|-------------|------|
| Emgu.CV | Cross platform .NET wrapper to OpenCV | [![NuGet version](https://badge.fury.io/nu/Emgu.CV.svg)](https://badge.fury.io/nu/Emgu.CV) |
| Emgu.CV.runtime.windows | Contains native runtime of Emgu CV for Windows | [![NuGet version](https://badge.fury.io/nu/Emgu.CV.runtime.windows.svg)](https://badge.fury.io/nu/Emgu.CV.runtime.windows) |

In [ ]:
// Import NuGet packages.
#r "nuget: Emgu.CV"
#r "nuget: Emgu.CV.runtime.windows"

// Import dependencies.
using System.Diagnostics;
using System.Drawing;
using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Structure;
using static System.Console;

# Emgu.CV Simple Webcam Example
---
**Description:** Opens a camera instance and closes once `ESC` button is pressed.

In [ ]:
using (var capture = new VideoCapture(0, VideoCapture.API.DShow))
using (var realtime = new Mat())
  while (CvInvoke.WaitKey(1) is not 27)
  {
    CvInvoke.NamedWindow("Realtime Capture");
    capture.Read(realtime);
    CvInvoke.Imshow("Realtime Capture", realtime);
  }

CvInvoke.DestroyAllWindows();

# Face and Eye Detection in Real Time
---
**Purpose:** Connect `Emgu.CV` to a real-time camera device.

**Description:** This code implements a VERY simple face detection software that uses [Haar Cascade algorithm](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html) to detect the frontal face and eyes. 

**Workflow:**
1. Instantiate a pre-trained Haar Cascade classifier.
2. Stream available camera using OpenCV on a window.
3. Modify frames to gray and prepare for classifier.
4. Classify according to trained cascades.
5. Listen to key to be pressed to close window.
6. Automatically dispose of all resources.

## Face Detection

In [ ]:
// Instantiate a pre-trained Haar Cascade classifier.
var faceCascade = new CascadeClassifier(@"Data/HaarCascadeFrontalFace.xml");
var eyeCascade = new CascadeClassifier(@"Data/HaarCascadeEyes.xml");

// Stream available camera using OpenCV on a window.
using (var capture = new VideoCapture(0, VideoCapture.API.DShow))
using (var grayImg = new Mat())
using (var realtime = new Mat())
  // Listen to key to be pressed to close window.
  while (CvInvoke.WaitKey(1) is not 27 && capture.IsOpened)
  {
    // Modify frames to gray and prepare for classifier.
    capture.Read(realtime);
    CvInvoke.CvtColor(realtime, grayImg, ColorConversion.Bgra2Gray);
    CvInvoke.EqualizeHist(grayImg, grayImg);

    // Classify according to trained cascades.
    var rectColor = new MCvScalar(0, 0, 255);
    foreach (var faceRect in faceCascade.DetectMultiScale(grayImg, minSize: new (30, 30)))
      CvInvoke.Rectangle(realtime, faceRect, rectColor);

    // Create window and show it.
    CvInvoke.NamedWindow("Realtime Capture");
    CvInvoke.Imshow("Realtime Capture", realtime);
  }

CvInvoke.DestroyAllWindows();

## Eye Detection

In [ ]:
// Instantiate a pre-trained Haar Cascade classifier.
var faceCascade = new CascadeClassifier(@"Data/HaarCascadeFrontalFace.xml");
var eyeCascade = new CascadeClassifier(@"Data/HaarCascadeEyes.xml");

// Stream available camera using OpenCV on a window.
using (var capture = new VideoCapture(0, VideoCapture.API.DShow))
using (var grayImg = new Mat())
using (var realtime = new Mat())
  // Listen to key to be pressed to close window.
  while (CvInvoke.WaitKey(1) is not 27 && capture.IsOpened)
  {
    // Modify frames to gray and prepare for classifier.
    capture.Read(realtime);
    CvInvoke.CvtColor(realtime, grayImg, ColorConversion.Bgra2Gray);
    CvInvoke.EqualizeHist(grayImg, grayImg);

    // Classify according to trained cascades.
    var rectColor = new MCvScalar(0, 0, 255);
    foreach (var faceRect in faceCascade.DetectMultiScale(grayImg, minSize: new (30, 30)))
      using (var detectedFace = new Mat(realtime, faceRect))
      using (var detectedFaceGray = new Mat())
      {
        CvInvoke.Rectangle(realtime, faceRect, rectColor);
        CvInvoke.CvtColor(detectedFace, detectedFaceGray, ColorConversion.Bgra2Gray);
        
        var circleColor = new MCvScalar(0, 255, 0);
        foreach (var eye in eyeCascade.DetectMultiScale(detectedFaceGray, minSize: new (30, 30)))
        {
          var x = (int)(Math.Round(eye.X + eye.Width * 0.5, MidpointRounding.ToEven) + faceRect.Left);
          var y = (int)(Math.Round(eye.Y + eye.Height * 0.5, MidpointRounding.ToEven) + faceRect.Top);
          var center = new Point(x, y);

          var radius = (int)(Math.Round((eye.Width + eye.Height) * 0.25, MidpointRounding.ToEven));
          CvInvoke.Circle(realtime, center, radius, circleColor, 2);
        }
      }

    // Create window and show it.
    CvInvoke.NamedWindow("Realtime Capture");
    CvInvoke.Imshow("Realtime Capture", realtime);
  }

CvInvoke.DestroyAllWindows();

## Facial Detection with FPS

In [ ]:
// Instantiate a pre-trained Haar Cascade classifier.
var faceCascade = new CascadeClassifier(@"Data/HaarCascadeFrontalFace.xml");
var eyeCascade = new CascadeClassifier(@"Data/HaarCascadeEyes.xml");

// Stream available camera using OpenCV on a window.
using (var capture = new VideoCapture(0, VideoCapture.API.DShow))
using (var grayImg = new Mat())
using (var realtime = new Mat())
  {
    // Add stopwatch to calculate FPS.
    var watch = new Stopwatch();
    watch.Start();
    var count = 0;

    // Listen to key to be pressed to close window.
    while (CvInvoke.WaitKey(1) is not 27 && capture.IsOpened)
    {
      count++;

      // Modify frames to gray and prepare for classifier.
      capture.Read(realtime);
      CvInvoke.CvtColor(realtime, grayImg, ColorConversion.Bgra2Gray);
      CvInvoke.EqualizeHist(grayImg, grayImg);

      // Classify according to trained cascades.
      var rectColor = new MCvScalar(0, 0, 255);
      foreach (var faceRect in faceCascade.DetectMultiScale(grayImg, minSize: new (30, 30)))
        using (var detectedFace = new Mat(realtime, faceRect))
        using (var detectedFaceGray = new Mat())
        {
          CvInvoke.Rectangle(realtime, faceRect, rectColor);
          CvInvoke.CvtColor(detectedFace, detectedFaceGray, ColorConversion.Bgra2Gray);
          
          var circleColor = new MCvScalar(0, 255, 0);
          foreach (var eye in eyeCascade.DetectMultiScale(detectedFaceGray, minSize: new (30, 30)))
          {
            var x = (int)(Math.Round(eye.X + eye.Width * 0.5, MidpointRounding.ToEven) + faceRect.Left);
            var y = (int)(Math.Round(eye.Y + eye.Height * 0.5, MidpointRounding.ToEven) + faceRect.Top);
            var center = new Point(x, y);

            var radius = (int)(Math.Round((eye.Width + eye.Height) * 0.25, MidpointRounding.ToEven));
            CvInvoke.Circle(realtime, center, radius, circleColor, 2);
          }
        }
        
        // Display FPS.
        var fpsColor = new MCvScalar(255, 0, 0);
        double frame = (double)count / watch.ElapsedMilliseconds * 1000;
        CvInvoke.PutText(realtime, $"FPS:{frame:F}", new (10, 30), FontFace.HersheyPlain, 2, fpsColor, 2);

      // Create window and show it.
      CvInvoke.NamedWindow("Realtime Capture");
      CvInvoke.Imshow("Realtime Capture", realtime);
    }
  }

CvInvoke.DestroyAllWindows();